In [9]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import KFold, StratifiedKFold
import warnings
import gc
import time
import sys
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

from catboost import Pool, CatBoostClassifier



In [4]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")


In [5]:
#CatBoost

idx = [c for c in train_df.columns if c not in ['ID_code', 'target']]
target = train_df['target']

df_list = [train_df, test_df]
new_df_list = []
new_train_df = pd.DataFrame()
new_train_df['target'] = train_df['target']
new_train_df['ID_code'] = train_df['ID_code']
new_test_df = pd.DataFrame()
new_test_df['ID_code'] = test_df['ID_code']
new_df_list = [new_train_df, new_test_df]
for i in range(0, len(df_list)):        
    for feat in idx:
        mean = df_list[i].loc[:,feat].mean()
        new_df_list[i][feat] = df_list[i][feat]
#         new_df_list[i]["mm_" + feat] = np.round(df_list[i][feat] - mean , 2) 
#         if mean > 0:
#             new_df_list[i]["is_more_" + feat] = np.where( df_list[i][feat] > mean, 10, -10 )
#         if mean < 0: 
#             new_df_list[i]["is_more_" + feat] = np.where( df_list[i][feat] < mean, 10, -10 )
            
features = [c for c in new_train_df.columns if c not in ['ID_code', 'target']]
target = new_train_df['target']

N_FOLDS = 5.0

model = CatBoostClassifier(loss_function="Logloss", eval_metric="AUC", iterations=2000, early_stopping_rounds= 500, l2_leaf_reg= 9, learning_rate= 0.1)
#model = CatBoostClassifier(eval_metric="AUC", depth=10, iterations= 1500, l2_leaf_reg= 9, learning_rate= 0.15)


kf = KFold(n_splits=int(N_FOLDS), random_state=2019, shuffle=True)

y_valid_pred = 0 * target
y_test_pred = 0
y_train_pred = 0 #storing results for ensemble

new_train_df.info()

for idx, (train_index, valid_index) in enumerate(kf.split(new_train_df)):
    y_train, y_valid = target.iloc[train_index], target.iloc[valid_index]
    X_train, X_valid = new_train_df[features].iloc[train_index,:], new_train_df[features].iloc[valid_index,:]
    _train = Pool(X_train, label=y_train)
    _valid = Pool(X_valid, label=y_valid)
    print( "\nFold ", idx)
    fit_model = model.fit(_train,
                          eval_set=_valid,
                          use_best_model=True,
                          verbose=200
                         )
    pred = fit_model.predict_proba(X_valid)[:,1]
    print( "  auc = ", roc_auc_score(y_valid, pred) )
    y_valid_pred.iloc[valid_index] = pred
    y_test_pred += fit_model.predict_proba(new_test_df[features])[:,1]
    y_train_pred += fit_model.predict_proba(new_train_df[features])[:,1]
y_test_pred /= N_FOLDS
y_train_pred /= N_FOLDS

#save an intermediate submission
sub_df1 = pd.DataFrame({"ID_code":test_df["ID_code"].values})
sub_df1["target"] = y_test_pred
sub_df1.to_csv("submission_catboost.csv", index=False)

ens_cat_df = pd.DataFrame({"ID_code":train_df["ID_code"].values})
ens_cat_df["output"] = y_train_pred
ens_cat_df.to_csv("../input/catboost_output.csv")



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 202 entries, target to var_199
dtypes: float64(200), int64(1), object(1)
memory usage: 308.2+ MB

Fold  0
0:	test: 0.5364710	best: 0.5364710 (0)	total: 199ms	remaining: 6m 38s
200:	test: 0.8776868	best: 0.8776868 (200)	total: 23.2s	remaining: 3m 28s
400:	test: 0.8900528	best: 0.8900528 (400)	total: 49.2s	remaining: 3m 16s
600:	test: 0.8947133	best: 0.8947133 (600)	total: 1m 12s	remaining: 2m 49s
800:	test: 0.8963380	best: 0.8963380 (800)	total: 1m 37s	remaining: 2m 25s
1000:	test: 0.8966581	best: 0.8967353 (993)	total: 2m	remaining: 2m
1200:	test: 0.8970510	best: 0.8970800 (1194)	total: 2m 23s	remaining: 1m 35s
1400:	test: 0.8970907	best: 0.8972818 (1294)	total: 2m 46s	remaining: 1m 11s
1600:	test: 0.8970987	best: 0.8972818 (1294)	total: 3m 8s	remaining: 47s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.8972817673
bestIteration = 1294

Shrink model to first 1295 iterations.
  a

In [6]:
#####LightGBM
param = {
            'num_leaves': 15, #was 10
            'max_bin': 119,
            'min_data_in_leaf': 11,
            'learning_rate': 0.01,
            'min_sum_hessian_in_leaf': 0.00245,
            'bagging_fraction': 1.0, 
            'bagging_freq': 5, 
            'feature_fraction': 0.05,
            'lambda_l1': 4.972,
            'lambda_l2': 50.0,   #30 #2.276,
            'min_gain_to_split': 0.65,
            'max_depth': 20, #17 #was 14
            'save_binary': True,
            'seed': 1337,
            'feature_fraction_seed': 1337,
            'bagging_seed': 1337,
            'drop_seed': 1337,
            'data_random_seed': 1337,
            'objective': 'binary',
            'boosting_type': 'gbdt',
            'verbose': 1,
            'metric': 'auc',
            'is_unbalance': True,
            'boost_from_average': True, #was false
        }
num_round = 15000
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=2019)
features = [c for c in train_df.columns if c not in ['ID_code', 'target']]
oof = np.zeros(len(train_df))
predictions = np.zeros(len(test_df))
train_preds = np.zeros(len(train_df))
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, target.values)):
    print("Fold {}".format(fold_))
    trn_data = lgb.Dataset(train_df.iloc[trn_idx][features], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(train_df.iloc[val_idx][features], label=target.iloc[val_idx])
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds = 200)
    oof[val_idx] = clf.predict(train_df.iloc[val_idx][features], num_iteration=clf.best_iteration)
    predictions += clf.predict(test_df[features], num_iteration=clf.best_iteration) / folds.n_splits
    train_preds += clf.predict(train_df[features], num_iteration=clf.best_iteration) / folds.n_splits
print("CV score: {:<8.5f}".format(roc_auc_score(target, oof)))

sub = pd.DataFrame({"ID_code": test_df.ID_code.values})
sub_lgb = pd.DataFrame({"ID_code": train_df.ID_code.values})
sub["target"] = predictions
sub_lgb["output"] = train_preds
sub.to_csv("submission_lgbm.csv", index=False)
sub_lgb.to_csv("../input/lgbm_output.csv")

Fold 0
Training until validation scores don't improve for 200 rounds.
[1000]	training's auc: 0.905752	valid_1's auc: 0.885235
[2000]	training's auc: 0.916553	valid_1's auc: 0.891313
[3000]	training's auc: 0.925128	valid_1's auc: 0.8953
[4000]	training's auc: 0.931935	valid_1's auc: 0.897273
[5000]	training's auc: 0.937554	valid_1's auc: 0.898575
[6000]	training's auc: 0.942476	valid_1's auc: 0.89927
[7000]	training's auc: 0.946842	valid_1's auc: 0.899761
Early stopping, best iteration is:
[6992]	training's auc: 0.946805	valid_1's auc: 0.899777
Fold 1
Training until validation scores don't improve for 200 rounds.
[1000]	training's auc: 0.905963	valid_1's auc: 0.881145
[2000]	training's auc: 0.916829	valid_1's auc: 0.887732
[3000]	training's auc: 0.925329	valid_1's auc: 0.891503
[4000]	training's auc: 0.93206	valid_1's auc: 0.894145
[5000]	training's auc: 0.937718	valid_1's auc: 0.895683
[6000]	training's auc: 0.942712	valid_1's auc: 0.896609
[7000]	training's auc: 0.947081	valid_1's auc

In [25]:
##NN Fitting
from keras.layers import (Flatten, Conv1D, Conv2D, Input, Dense, Dropout, BatchNormalization,
                          concatenate, GaussianNoise, Reshape, TimeDistributed, LeakyReLU, PReLU, Embedding)
from keras.models import Model, load_model, save_model
from keras.optimizers import SGD, Adam
from sklearn.base import BaseEstimator, ClassifierMixin
from pathlib import Path
from keras.callbacks import Callback


In [42]:
feats = ["var_{}".format(i) for i in range(200)]
targets2 = ["target{}".format(i) for i in range(0,2)]

split_tar_df = train_df.copy(deep=True)
split_tar_df['target1'] = train_df['target']
split_tar_df['target0'] = 1 - train_df['target']

X = train_df[feats]
X_test = test_df[feats]

y = split_tar_df[['target1','target0']]

N_FOLDS = 5.0

kf = KFold(n_splits=int(N_FOLDS), random_state=2019, shuffle=True)

#cvlist = list(StratifiedKFold(5, random_state=12345786).split(X, y))
scaler = StandardScaler()

X_sc = scaler.fit_transform(X)
X_test_sc = scaler.fit_transform(X_test)

In [43]:
class ROC_AUC(Callback):
    def __init__(self, validation_data):
        self.X_val, self.y_val = validation_data
    
    def on_epoch_end(self, epoch, logs={}):
        print("ROC AUC for this fold is ", roc_auc_score(self.y_val, self.model.predict(X_val)))
        
class NNv1(BaseEstimator, ClassifierMixin):
    def __init__(self,
                 inp_shape=200,
                 gaussian_noise=0.01,
                 dense1_dim=32,
                 dense2_dim=32,
                 dense1_kwargs=None,
                 dense2_kwargs=None,
                 classifier_kwargs=None,
                 optimizer=SGD,
                 opt_kwargs=None,
                 ):
        self.inp_shape = inp_shape
        self.gaussian_noise = gaussian_noise
        self.dense1_dim = dense1_dim
        self.dense2_dim = dense2_dim
        self.dense1_kwargs = dense1_kwargs
        self.dense2_kwargs = dense2_kwargs
        self.classifier_kwargs = classifier_kwargs
        self.optimizer = optimizer
        self.opt_kwargs = opt_kwargs
        self._default_initiaization()

    def _default_initiaization(self):
        if self.dense1_kwargs is None:
            self.dense1_kwargs = {"kernel_initializer": "glorot_uniform"}
        if self.dense2_kwargs is None:
            self.dense2_kwargs = {"kernel_initializer": "he_uniform"}
        if self.classifier_kwargs is None:
            self.classifier_kwargs = {"kernel_initializer": "he_uniform"}
        if self.opt_kwargs is None:
            self.opt_kwargs = {}

    def _build_model(self):
        inp = Input(shape=(self.inp_shape,))
        # x = GaussianNoise(self.gaussian_noise)(inp)
        x = Reshape((self.inp_shape, 1))(inp)
        d1 = Dense(self.dense1_dim, activation='tanh',)(x)
        #d1 = TimeDistributed(Dropout(0.2))(d1)
        d2 = Dense(self.dense1_dim, activation='relu',)(x)
        #d2 = PReLU()(d2)
        #d2 = TimeDistributed(Dropout(0.2))(d2)
        x = concatenate([d1, d2])
        x = Flatten()(x)
        out = Dense(2, activation='softmax', **self.classifier_kwargs)(x)

        model = Model(inputs=inp, outputs=out)
        opt = self.optimizer(**self.opt_kwargs)
        model.compile(loss='binary_crossentropy', optimizer=opt)
        return model

    def fit(self, X, y, *args, **kwargs):
        self.model = self._build_model()
        print(self.model.summary())
        self.model.fit(X, y, *args, **kwargs)
        return self

    def predict(self, X, y=None, weight_path=None, **kwargs):
        if self.model:
            if weight_path is not None:
                self.model.load_weights(weight_path)
            y_hat = self.model.predict(X, **kwargs)
        else:
            raise ValueError("Model not fit yet")
        return y_hat

In [48]:
model = NNv1(opt_kwargs = {"lr": 0.01, "momentum": 0.9, "nesterov": True, "clipnorm": 1})
y_preds_nn = np.zeros((len(y)))

n = 0
for idx, (tr_idx, val_idx) in enumerate(kf.split(X_sc)):    
#for tr_idx, val_idx in cvlist:
    print("Fold {}".format(n))
    n += 1
    X_dev, y_dev = X_sc[tr_idx], y.iloc[tr_idx]
    X_val, y_val = X_sc[val_idx], y.iloc[val_idx]
    roc_auc = ROC_AUC((X_val, y_val))    
    model.fit(X_dev, y_dev, validation_data=(X_val, y_val), epochs=20, batch_size=256, verbose=0, callbacks=[roc_auc])
    val_preds = model.predict(X_val, batch_size=5000)
    y_preds_nn[val_idx] = val_preds[:,0].flatten()
    
sub = pd.DataFrame({"ID_code": test_df.ID_code.values})
output_df = pd.DataFrame({"ID_code": train_df.ID_code.values})
y_test_preds = model.predict(X_test_sc, batch_size = 5000)
y_train_preds = model.predict(X_sc, batch_size = 5000)
sub['target'] = y_test_preds[:,0].flatten()
output_df['output'] = y_train_preds[:,0].flatten()
sub.to_csv('submission_full_nn.csv', index=False)
output_df.to_csv('../input/full_nn_output.csv')

Fold 0
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           (None, 200)          0                                            
__________________________________________________________________________________________________
reshape_21 (Reshape)            (None, 200, 1)       0           input_21[0][0]                   
__________________________________________________________________________________________________
dense_73 (Dense)                (None, 200, 32)      64          reshape_21[0][0]                 
__________________________________________________________________________________________________
dense_74 (Dense)                (None, 200, 32)      64          reshape_21[0][0]                 
__________________________________________________________________________________________________
con

ROC AUC for this fold is  0.8977507767668198
ROC AUC for this fold is  0.8981411821456096
ROC AUC for this fold is  0.8981923712957003
ROC AUC for this fold is  0.8985029405124682
ROC AUC for this fold is  0.8983535988471616
ROC AUC for this fold is  0.8987801849118986
ROC AUC for this fold is  0.8987339264215605
ROC AUC for this fold is  0.8989264039992164
Fold 3
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_24 (InputLayer)           (None, 200)          0                                            
__________________________________________________________________________________________________
reshape_24 (Reshape)            (None, 200, 1)       0           input_24[0][0]                   
__________________________________________________________________________________________________
dense_82 (Dense)                (None, 

In [49]:
##NN Ensemble 
from keras import models
from keras import layers
from keras import optimizers

from sklearn.utils import shuffle

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(3,)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(2, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

print(model.summary())

N_FOLDS = 5.0

kf = KFold(n_splits=int(N_FOLDS), random_state=2019, shuffle=True)





_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_88 (Dense)             (None, 256)               1024      
_________________________________________________________________
dropout_9 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_89 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_10 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_90 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_11 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_91 (Dense)             (None, 128)               8320      
__________

In [50]:
#Load previous data

catb_df = pd.read_csv("../input/catboost_output.csv")
lgbm_df = pd.read_csv("../input/lgbm_output.csv")
nn_df = pd.read_csv("../input/full_nn_output.csv")

submission_catb_df = pd.read_csv("submission_catboost.csv")
submission_lgbm_df = pd.read_csv("submission_lgbm.csv")
submission_full_nn_df = pd.read_csv("submission_full_nn.csv")

combined_df = pd.DataFrame({"ID_code": catb_df.ID_code.values})
combined_df['cat_out'] = catb_df['output']
combined_df['lgbm_out'] = lgbm_df['output']
combined_df['full_nn_out'] = nn_df['output']
combined_df['target1'] = train_df['target']
combined_df['target0'] = 1 - train_df['target']

#drop 90% of 0 scores to make balanced input for NN
#combined_df = combined_df.drop(combined_df[combined_df['target0'] > 0.9].sample(frac=0.1).index)
zero_subset_df = combined_df[combined_df['target0'] > 0.9].sample(frac = 0.1)
ones_subset_df = combined_df[combined_df['target1'] > 0.9]

frames = [zero_subset_df, ones_subset_df]

combined_df = pd.concat(frames)
combined_df = shuffle(combined_df)

combined_test_df = pd.DataFrame({"ID_code": test_df.ID_code.values})
combined_test_df['cat_out'] = submission_catb_df['target']
combined_test_df['lgbm_out'] = submission_lgbm_df['target']
combined_test_df['full_nn_out'] = submission_full_nn_df['target']

zero_subset_df.to_csv('../input/zeros_sample.csv')
ones_subset_df.to_csv('../input/ones_sample.csv')
combined_df.to_csv('../input/combined.csv')

#idx = [c for c in combined_df.columns if c not in ['ID_code', 'target']]
features = [c for c in combined_df.columns if c not in ['ID_code', 'target', 'target1', 'target0']]
target = combined_df.loc[:, ['target1', 'target0']]

y_test_pred = np.zeros(len(test_df))
    
for idx, (train_index, valid_index) in enumerate(kf.split(combined_df)):
    print("Fold {}".format(idx))
    y_train, y_valid = target.iloc[train_index], target.iloc[valid_index]
    X_train, X_valid = combined_df[features].iloc[train_index,:], combined_df[features].iloc[valid_index,:]
    model.fit(X_train, y_train, epochs=20, batch_size = 512, validation_data =(X_valid, y_valid))
    pred = model.predict_proba(X_valid)
    print(pred.shape)
    print( "  auc = ", roc_auc_score(y_valid, pred) )
    y_test_pred += model.predict_proba(combined_test_df[features])[:,0]
    print(y_test_pred.shape)

#save base submission
sub_df1 = pd.DataFrame({"ID_code":test_df["ID_code"].values})
sub_df1["target"] = y_test_pred / N_FOLDS
sub_df1.to_csv("submission_nn.csv", index=False)
#make combined data set for comparison
combined_test_df['nn_out'] = y_test_pred / N_FOLDS
combined_test_df.to_csv("../input/combined_output.csv")


Fold 0
Train on 30470 samples, validate on 7618 samples
Epoch 1/20
30470/30470 [==============================] - 2s 61us/step - loss: 0.3703 - acc: 0.8521 - val_loss: 0.2444 - val_acc: 0.9161
Epoch 2/20
30470/30470 [==============================] - 0s 12us/step - loss: 0.2217 - acc: 0.9128 - val_loss: 0.1731 - val_acc: 0.9319
Epoch 3/20
30470/30470 [==============================] - 0s 12us/step - loss: 0.2005 - acc: 0.9222 - val_loss: 0.1705 - val_acc: 0.9304
Epoch 4/20
30470/30470 [==============================] - 0s 13us/step - loss: 0.1863 - acc: 0.9278 - val_loss: 0.1974 - val_acc: 0.9279
Epoch 5/20
30470/30470 [==============================] - 0s 12us/step - loss: 0.1792 - acc: 0.9293 - val_loss: 0.2268 - val_acc: 0.9132
Epoch 6/20
30470/30470 [==============================] - 0s 12us/step - loss: 0.1776 - acc: 0.9306 - val_loss: 0.1575 - val_acc: 0.9369
Epoch 7/20
30470/30470 [==============================] - 0s 13us/step - loss: 0.1719 - acc: 0.9337 - val_loss: 0.1542 - v

PermissionError: [Errno 13] Permission denied: '../input/combined_output.csv'